In [363]:
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
import tensorflow as tf
import os

In [364]:
train='/home/zhangbaojun/TrafficFlowPrediction-tensorflow/data/100211_all_train.csv'
test='/home/zhangbaojun/TrafficFlowPrediction-tensorflow/data/100211_all_test.csv'
train_data=pd.read_csv(train).fillna(0)
test_data=pd.read_csv(test).fillna(0)

MinMaxScaler（最小最大值标准化）
　　公式：1、X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
         2、X_scaler = X_std/ (max - min) + min

In [365]:
minmax=preprocessing.MinMaxScaler()
train_minmax=minmax.fit_transform(train_data["volumn"].values.reshape(-1,1))
test_minmax=minmax.fit_transform(test_data["volumn"].values.reshape(-1,1))

/home/zhangbaojun/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [366]:
train=[]
test=[]
for i in range(12, len(train_minmax)):
    train.append(np.float32(train_minmax[i - 12: i + 1]))
for i in range(12,len(test_minmax)):
    test.append(np.float32(test_minmax[i-12:i+1]))

In [371]:
train=np.array(train)
test=np.array(test)
#np.random.shuffle(train)
X_train=train[:,0:-1]
Y_train=train[:,-1]
X_test=test[:,0:-1]
Y_test=test[:,-1]
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))


17474
17474
9153
9153


TFrecords generate

In [ ]:
train_tfrecords='/home/zhangbaojun/TrafficFlowPrediction-tensorflow/data/train_tfrecords'
test_tfrecords='/home/zhangbaojun/TrafficFlowPrediction-tensorflow/data/test_tfrecords'

train_writer=tf.python_io.TFRecordWriter(path=train_tfrecords)
for i in range(0,len(X_train)):
    features1 = tf.train.Features(
        feature={
            'features':tf.train.Feature(bytes_list=tf.train.BytesList(value=[X_train[i].tostring()])),
            'label': tf.train.Feature(float_list=tf.train.FloatList(value=[Y_train[i]]))})
    example=tf.train.Example(features=features1)
    train_writer.write(example.SerializeToString())
    print(X_train[i],Y_train[i])
train_writer.close()

test_write=tf.python_io.TFRecordWriter(path=test_tfrecords)
for j in range(0,len(X_test)):
    features2 = tf.train.Features(
        feature = {
            'features':tf.train.Feature(bytes_list=tf.train.BytesList(value=[X_test[j].tostring()])),
            'label':tf.train.Feature(float_list=tf.train.FloatList(value=[Y_test[j]]))})
    example=tf.train.Example(features=features2)
    test_write.write(example.SerializeToString())
test_write.close()